# pandapower-Netze modellieren

Dieses Tutorial enthält eine Einführung in die pandapower-Datenstrukturen und zeigt, wie man mit Hilfe der pandapower-API ein Netzwerk modelliert. Das folgende Beispiel demonstriert den Einsatz der gängigsten Komponenten, die in pandapower verfügbar sind.

<img src="pics/example_network_simple.png">

Die Datenstruktur der pandapower-Umgebung basiert auf der Python-Bibliothek pandas. Ein pandapower-Netzwerk besteht aus einer separaten Komponententabelle für jeden Komponententyp, der im Netzwerk eingesetzt wird. Tabellenspalten repräsentieren die Parameter der jeweiligen Komponente. 

Durch das Ausführen der folgenden Code-Zellen generieren Sie verschiedene Komponententabellen. Weitere Informationen zu den verwendeten Komponenten finden Sie in der pandapower-Dokumentation im Abschnitt "Datastructures and Elements".


### Leeres Netzwerk

Zunächst wird das pandapower-Modul importiert und anschließend ein leerer Netzcontainer erzeugt:

In [ ]:
import pandapower as pp #import pandapower

net = pp.create_empty_network() #create an empty network

### Buses

<img src="pics/example_network_simple_buses.png">

Nun werden die Netzknoten generiert, die in pandapower als "buses" bezeichnet werden. Weitere Eingabeparameter sind die Nennspannungen an den Knoten (vn_kv), die gleichzeitig den Startwert für den zur Berechnung eingesetzten Newton-Raphson-Algorithmus darstellen. Ein Name kann zur besseren Lesbarkeit angegeben werden.

Insgesamt werden drei Hochspannungsknoten (vn_kv=110.) und vier Mittelspannungsknoten (vn_kv=20.) benötigt. 

In [ ]:
bus1 = pp.create_bus(net, name="HV Busbar", vn_kv=110, type="b")
bus2 = pp.create_bus(net, name="HV Busbar 2", vn_kv=110, type="b")
bus3 = pp.create_bus(net, name="HV Transformer Bus", vn_kv=110, type="n")
bus4 = pp.create_bus(net, name="MV Transformer Bus", vn_kv=20, type="n")
bus5 = pp.create_bus(net, name="MV Main Bus", vn_kv=20, type="b")
bus6 = pp.create_bus(net, name="MV Bus 1", vn_kv=20, type="b")
bus7 = pp.create_bus(net, name="MV Bus 2", vn_kv=20, type="b")

Bus 3 und Bus 4 wurden als "nodes" klassifiziert (type="n"), alle anderen werden als "busbars" eingestuft (type="b"):

In [ ]:
net.bus # show bus table

Alle create-Funktionen geben den pandapower-Index der erzeugten Komponente zurück. Variable bus6 entspricht damit dem Index des buses mit dem Namen "MV Bus 1" (welcher den Index 5 hat):


In [ ]:
bus6

Wir nutzen diese Variablen, um weitere bus- und branch-Elemente hinzuzufügen.

### External Grid

<img src="pics/example_network_simple_ext_grid.png">

Jetzt wird eine External Grid-Verbindung erstellt, welche als slack-Knoten für die Lastflussberechnung fungiert. Die Spannung der External-Grid-Komponente wird auf 1.02 per unit gesetzt. Des Weiteren definieren wir einen Phasenwinkel von 50 Grad:

In [ ]:
pp.create_ext_grid(net, bus1, vm_pu=1.02, va_degree=50) # Create an external grid connection

net.ext_grid #show external grid table

### Transformatoren 

<img src="pics/example_network_simple_trafo.png">

Der Transformator verbindet die Hochspannungs- mit der Mittelspannungsseite des Netzwerks. Der Hochspannungsbus des Transformators wird mit Bus 3 verbunden, während der Mittelspannungsbus mit Bus 4 verbunden ist. Wir wählen einen Transformatortyp aus der in pandapower enthaltenen Standardtypenbibliothek aus: 25 MVA 110/20 kV.


In [ ]:
trafo1 = pp.create_transformer(net, bus3, bus4, name="110kV/20kV transformer", std_type="25 MVA 110/20 kV")

Die detaillierten Trafo-Parameter, wie z.B. die Kurzschlussspannung, Nennleistung oder Eisenverluste, werden automatisch aus der Bibliothek geladen und in der Trafo-Tabelle gespeichert: 


In [ ]:
net.trafo #show transformer table

### Verbindungen

Das Netzwerk beinhaltet drei Mittelspannungsverbindungen und eine Hochspannungsverbindung. Die Bus-Anbindungen und Kabellängen sind im Diagramm angegeben:

<img src="pics/example_network_simple_lines.png">

Die Verbindungsparameter werden erneut aus der Standardbibliothek entnommen. Wir nutzen unterschiedliche Kabellängen und Standardtypen für jedes Kabel:

In [ ]:
line1 = pp.create_line(net, bus1, bus2, length_km=10, std_type="N2XS(FL)2Y 1x300 RM/35 64/110 kV",  name="Line 1")
line2 = pp.create_line(net, bus5, bus6, length_km=2.0, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 2")
line3 = pp.create_line(net, bus6, bus7, length_km=3.5, std_type="48-AL1/8-ST1A 20.0", name="Line 3")
line4 = pp.create_line(net, bus7, bus5, length_km=2.5, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 4")

Anschließend sieht die Tabelle wie folgt aus:

In [ ]:
net.line # show line table

### Schalter

Es gibt zwei Schalter, welche den Stromkreis unterbrechen können (je einen auf Hoch- und Mittelspannungsebene). Die Schalter sind zwischen Bus 4 und Bus 5 bzw. zwischen Bus 2 und Bus 3 platziert. Ein zwischen zwei Bussen eingesetzter Schalter wird mit dem Parameter et="b" definiert.

<img src="pics/example_network_simple_switches.png">

In [ ]:
sw1 = pp.create_switch(net, bus2, bus3, et="b", type="CB", closed=True)
sw2 = pp.create_switch(net, bus4, bus5, et="b", type="CB", closed=True)

Weiterhin werden alle Bus/Line-Verbindungen im Mittelspannungsnetz mit load-break-Schaltern versehen ("LBS"). Diese Art Schalter wird mit dem Parameter et="l" versehen.


In [ ]:
sw3 = pp.create_switch(net, bus5, line2, et="l", type="LBS", closed=True)
sw4 = pp.create_switch(net, bus6, line2, et="l", type="LBS", closed=True)
sw5 = pp.create_switch(net, bus6, line3, et="l", type="LBS", closed=True)
sw6 = pp.create_switch(net, bus7, line3, et="l", type="LBS", closed=False)
sw7 = pp.create_switch(net, bus7, line4, et="l", type="LBS", closed=True)
sw8 = pp.create_switch(net, bus5, line4, et="l", type="LBS", closed=True)

Die Tabelle der Schalter enthält nun alle definierten Komponenten. Die Spalte "bus" enthält den Index des Busses, an den der Schalter angeschlossen ist. Für Schalter des b-Typs enthält die Tabelle den Index des zweiten Buses. Im Falle des l-Typs enthält die Tabelle den Index der Verbindung, an welche der Schalter angeschlossen ist. Alle Schalter sind geschlossen.


In [ ]:
net.switch # show switch table

### Lasten

<img src="pics/example_network_simple_load.png">

Das Last-Element wird genutzt um aktive und reaktve Verbraucher zu modellieren. Es wird eine 2 MW / 3MVar-Last mit einem Skalierungsfaktor von 0.6 erzeugt.


In [ ]:
pp.create_load(net, bus7, p_mw=2, q_mvar=4, scaling=0.6, name="load")

net.load

#### Spannungsabhängige Lasten - ZIP-Lastmodell

Man kann die Parameter `const_z_percent` und `const_i_percent` bei Lasten anpassen, die allerdings standardmäßig zu 0 gesetzt sind. Sie können eingesetzt werden, um einen spannungsabhängigen Lastanteil zu definieren. Dazu wird ein sogenanntes **ZIP load**-Modell verwendet, welches die Last als Kombination aus konstanter Leistung, konstantem Strom und konstanter Impedanz beschreibt.

Exemplarisch wird eine 2 MW/4 MVar-Last mit 30% konstanter Impedanz und 20% konstantem Strom erzeugt:

In [ ]:
pp.create_load(net, bus7, p_mw=2, q_mvar=4, const_z_percent=30, const_i_percent=20, name="zip_load")

net.load

### Statischer Generator

<img src="pics/example_network_simple_sgen.png">

Der statische Generator wird genutzt, um aktive und reaktive Generatoren mit konstanter Leistung zu modellieren. Weil das Vorzeichensystem von pandapower immer die Sicht des Verbrauchers beschreibt, ist die aktive Leistung negativ, wenn Leistung erzeugt wird. Im Beispiel wird ein statischer Generator mit einer Leistung von 2 MW / 500 kVar genutzt.


In [ ]:
pp.create_sgen(net, bus7, p_mw=2, q_mvar=-0.5, name="static generator")

net.sgen

### Spannungsgeregelter Generator

<img src="pics/example_network_simple_gen.png">

Die Generator-Komponente wird genutzt, um die konstante, spannungsgeregelte Generierung von Leistung zu modellieren. Der Generator wird mit einem Arbeitspunkt für die Spannung definiert:


In [ ]:
pp.create_gen(net, bus6, p_mw=6, max_q_mvar=3, min_q_mvar=-3, vm_pu=1.03, name="generator") 

net.gen

### Shunt


<img src="pics/example_network_simple_shunt.png">

Ein shunt wird über dessen aktiven und reaktiven Verbrauch bei der Nennleistung charakterisiert. Zur Erinnerung: Das Vorzeichensystem basiert auf der Verbrauchersicht. Es wird eine Kapazitätsbank modelliert, weshalb die definierte reaktive Last negativ ist:
 

In [ ]:
pp.create_shunt(net, bus3, q_mvar=-0.96, p_mw=0, name='Shunt')

net.shunt

Die Berechnung kann jetzt gestartet werden:

In [ ]:
pp.runpp(net)

Nach der Berechnung werden dem Netz-Container Ergebnistabellen hinzugefügt, die wie folgt abgerufen werden können:

In [ ]:
net.res_line

### Lessons learned

Nach dem Durcharbeiten des Tutorials sollten Sie wissen:
    
- welche Schritte vor einer Berechnung durchzuführen sind
- welche Komponenten von pandapower zur Verfügung gestellt werden
- dass es eine Komponentenbibliothek in pandapower gibt
- mit welchem Befehl eine Berechnung gestartet wird und
- wie man Ergebnisse auswertet